# Walkthrough Method Testing

**Goal**: Test the weighting and hard cut config of the data loading process

In [1]:
%load_ext autoreload
%autoreload 2

import os
import yaml
import logging

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import torch
from torch_geometric.data import Data
import matplotlib.pyplot as plt
from itertools import chain

logging.basicConfig(level=logging.INFO)

/global/homes/d/danieltm/.conda/envs/gnn4itk/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data load

In [2]:
input_dir = "/global/cfs/cdirs/m3443/data/GNN4ITK/CommonFrameworkExamples/Example_1_Dev/gnn/valset/"
input_files = os.listdir(input_dir)
input_files = [os.path.join(input_dir, f) for f in input_files]

In [3]:
sample = torch.load(input_files[0])

In [4]:
sample

Data(hit_id=[278183], x=[278183], y=[34671], z=[278183], r=[278183], phi=[278183], eta=[278183], region=[278183], cluster_x_1=[278183], cluster_y_1=[278183], cluster_z_1=[278183], cluster_x_2=[278183], cluster_y_2=[278183], cluster_z_2=[278183], norm_x=[278183], norm_y=[278183], norm_z_1=[278183], eta_angle_1=[278183], phi_angle_1=[278183], eta_angle_2=[278183], phi_angle_2=[278183], norm_z_2=[278183], track_edges=[2, 14704], particle_id=[14704], pt=[14704], radius=[14704], primary=[14704], nhits=[14704], pdgId=[14704], config=[2], event_id='000000102', edge_index=[2, 34671], truth_map=[14704], weights=[34671], scores=[34671])

In [5]:
score_cut = 0.8
true = sample.y.sum()
positive = (sample.scores > score_cut).sum()
true_positive = ((sample.scores > score_cut) & sample.y).sum()

eff = true_positive / true
pur = true_positive / positive

print("Efficiency: {:.2f}%, Purity: {:.2f}%".format(eff*100, pur*100))

Efficiency: 97.40%, Purity: 98.43%


## Algorithm Dev

### Version 1: NetworkX All Paths, Naive Implementation

In [6]:
# Need to...
# 1. Apply first score cut
# 2. Using networkx, find all paths between all starting nodes and ending nodes

In [7]:
# Use pytorch geometric to convert to networkx
import networkx as nx
import torch_geometric
from torch_geometric.utils import to_networkx

In [8]:
# Apply score cut
edge_mask = sample.scores > score_cut
sample.edge_index = sample.edge_index[:, edge_mask]
sample.y = sample.y[edge_mask]
sample.scores = sample.scores[edge_mask]

# Convert to networkx graph
G = to_networkx(sample, to_undirected=False)

In [9]:
# Get some topline stats, starting nodes are nodes with no incoming edges, ending nodes are nodes with no outgoing edges
print(f"Number of nodes: {G.number_of_nodes()} \nNumber of edges: {G.number_of_edges()} \nNumber starting nodes: {len([n for n in G.nodes() if G.in_degree(n) == 0])} \nNumber ending nodes: {len([n for n in G.nodes() if G.out_degree(n) == 0])}")

Number of nodes: 278183 
Number of edges: 13817 
Number starting nodes: 264483 
Number ending nodes: 264473


In [10]:
# Remove isolated nodes
G.remove_nodes_from(list(nx.isolates(G)))

In [11]:
# Get some topline stats, starting nodes are nodes with no incoming edges, ending nodes are nodes with no outgoing edges
print(f"Number of nodes: {G.number_of_nodes()} \nNumber of edges: {G.number_of_edges()} \nNumber starting nodes: {len([n for n in G.nodes() if G.in_degree(n) == 0])} \nNumber ending nodes: {len([n for n in G.nodes() if G.out_degree(n) == 0])}")

Number of nodes: 15175 
Number of edges: 13817 
Number starting nodes: 1475 
Number ending nodes: 1465


In [89]:
starting_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
ending_nodes = [n for n in G.nodes() if G.out_degree(n) == 0]

# Get all paths between starting and ending nodes
paths = []
for start in tqdm(starting_nodes):
    paths.extend(
        list(nx.all_simple_paths(G, start, end))
        for end in ending_nodes
        if nx.has_path(G, start, end)
    )
# Get the number of paths by expanding out the path list of lists of lists
paths = list(chain.from_iterable(paths))
num_paths = len(paths)
print(f"Number of paths: {num_paths}")

100%|██████████| 1475/1475 [00:19<00:00, 74.82it/s] 

Number of paths: 1586


In [18]:
%%time
starting_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
ending_nodes = [n for n in G.nodes() if G.out_degree(n) == 0]

# Get shortest path between starting and ending nodes
paths = []
for start in tqdm(starting_nodes):
    paths.extend(
        list(nx.shortest_path(G, start, end))
        for end in ending_nodes
        if nx.has_path(G, start, end)
    )

# Get the number of paths
num_paths = len(paths)
print(f"Number of paths: {num_paths}")

100%|██████████| 1475/1475 [00:19<00:00, 75.29it/s] 

Number of paths: 1523
CPU times: user 19.7 s, sys: 30.1 ms, total: 19.7 s
Wall time: 19.6 s


In [17]:
%%time
starting_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
ending_nodes = [n for n in G.nodes() if G.out_degree(n) == 0]

# Get shortest path between starting and ending nodes using the A star function
paths = []
for start in tqdm(starting_nodes):
    paths.extend(
        list(nx.astar_path(G, start, end))
        for end in ending_nodes
        if nx.has_path(G, start, end)
    )

# Get the number of paths
num_paths = len(paths)
print(f"Number of paths: {num_paths}")

100%|██████████| 1475/1475 [00:20<00:00, 72.15it/s] 

Number of paths: 1523
CPU times: user 20.5 s, sys: 0 ns, total: 20.5 s
Wall time: 20.5 s


In [20]:
%%time
starting_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
ending_nodes = [n for n in G.nodes() if G.out_degree(n) == 0]

# Get shortest path between starting and ending nodes using the A star function
paths = []
for start in tqdm(starting_nodes):
    paths.extend(
        list(nx.bidirectional_shortest_path(G, start, end))
        for end in ending_nodes
        if nx.has_path(G, start, end)
    )

# Get the number of paths
num_paths = len(paths)
print(f"Number of paths: {num_paths}")

100%|██████████| 1475/1475 [00:17<00:00, 85.82it/s] 

Number of paths: 1523
CPU times: user 17.2 s, sys: 21.7 ms, total: 17.2 s
Wall time: 17.2 s


### Version 2: Multithreaded NetworkX All Paths, Naive Implementation

In [6]:
# Need to...
# 1. Apply first score cut
# 2. Using networkx, find all paths between all starting nodes and ending nodes

In [6]:
# Use pytorch geometric to convert to networkx
import networkx as nx
import torch_geometric
from torch_geometric.utils import to_networkx
from concurrent.futures import ThreadPoolExecutor, as_completed

In [7]:
# Apply score cut
edge_mask = sample.scores > score_cut
sample.edge_index = sample.edge_index[:, edge_mask]
sample.y = sample.y[edge_mask]
sample.scores = sample.scores[edge_mask]

# Convert to networkx graph
G = to_networkx(sample, to_undirected=False)
G.remove_nodes_from(list(nx.isolates(G)))

In [8]:
# Get some topline stats, starting nodes are nodes with no incoming edges, ending nodes are nodes with no outgoing edges
print(f"Number of nodes: {G.number_of_nodes()} \nNumber of edges: {G.number_of_edges()} \nNumber starting nodes: {len([n for n in G.nodes() if G.in_degree(n) == 0])} \nNumber ending nodes: {len([n for n in G.nodes() if G.out_degree(n) == 0])}")

Number of nodes: 15175 
Number of edges: 13817 
Number starting nodes: 1475 
Number ending nodes: 1465


In [9]:
def find_shortest_paths(start):
    return [
        nx.shortest_path(G, start, end)
        for end in ending_nodes
        if nx.has_path(G, start, end)
    ]

In [10]:
def find_all_paths(start):
    return list(chain.from_iterable([
        list(nx.all_simple_paths(G, start, end))
        for end in ending_nodes
        if nx.has_path(G, start, end)
    ]))

In [11]:
%%time

# Do the above, but using tqdm pool
from tqdm.contrib.concurrent import process_map

starting_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
ending_nodes = [n for n in G.nodes() if G.out_degree(n) == 0]

workers = 32

paths = process_map(find_all_paths, starting_nodes, max_workers=workers, chunksize=1)

# Get the number of paths
num_paths = len(list(chain.from_iterable(paths)))
print(f"Number of paths: {num_paths}")

100%|██████████| 1475/1475 [00:00<00:00, 1983.97it/s]

Number of paths: 1586
CPU times: user 477 ms, sys: 459 ms, total: 936 ms
Wall time: 1.52 s


In [12]:
starting_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
ending_nodes = [n for n in G.nodes() if G.out_degree(n) == 0]

workers = 32

# Use process pool
import multiprocessing
from multiprocessing import Pool

with Pool(workers) as p:
    paths = p.map(find_all_paths, starting_nodes)

# Get the number of paths
num_paths = len(list(chain.from_iterable(paths)))
print(f"Number of paths: {num_paths}")

Number of paths: 1586


In [14]:
def find_all_paths(start, G, ending_nodes):
    return list(chain.from_iterable([
        list(nx.all_simple_paths(G, start, end))
        for end in ending_nodes
        if nx.has_path(G, start, end)
    ]))

In [18]:
from functools import partial

starting_nodes = [n for n in G.nodes() if G.in_degree(n) == 0]
ending_nodes = [n for n in G.nodes() if G.out_degree(n) == 0]

workers = 32

# Use process pool
import multiprocessing
from multiprocessing import Pool

find_all_paths_partial = partial(find_all_paths, G=G, ending_nodes=ending_nodes)

with Pool(workers) as p:
    paths = p.map(find_all_paths_partial, starting_nodes)

    

### Using GPU

In [61]:
# Import cugraph and use it in place of networkx
import cugraph
import cudf

## Evaluation

In [ ]:
# Create a dataframe that labels each node with the path it is in
track_df = pd.DataFrame(
    {
        "hit_id": list(chain.from_iterable(paths)),
        "track_id": list(chain.from_iterable([[i] * len(p) for i, p in enumerate(paths)])),
    }
)

In [ ]:
# Remove duplicates on hit_id
track_df = track_df.drop_duplicates(subset="hit_id")

In [ ]:
hit_id = track_df.hit_id
track_id = track_df.track_id

In [ ]:
track_id_tensor = torch.ones(len(sample.x), dtype=torch.long) * -1
track_id_tensor[hit_id.values] = torch.from_numpy(track_id.values)

In [ ]:
sample.labels = track_id_tensor

### Current Evaluation Methods

In [18]:
config = yaml.safe_load(open("track_building_eval.yaml", "r"))

In [19]:
from gnn4itk_cf.stages.track_building import utils 

INFO:root:Wandb found, using WandbLogger


In [69]:
def load_reconstruction_df(graph):
    """Load the reconstructed tracks from a file."""
    pids = torch.zeros(graph.hit_id.shape[0], dtype=torch.int64)
    pids[graph.track_edges[0]] = graph.particle_id
    pids[graph.track_edges[1]] = graph.particle_id

    return pd.DataFrame({"hit_id": graph.hit_id, "track_id": graph.labels, "particle_id": pids})

def load_particles_df(graph):
    """Load the particles from a file."""
    # Get the particle dataframe
    particles_df = pd.DataFrame({"particle_id": graph.particle_id, "pt": graph.pt})

    # Reduce to only unique particle_ids
    particles_df = particles_df.drop_duplicates(subset=['particle_id'])

    return particles_df

def get_matching_df(reconstruction_df, min_track_length=1, min_particle_length=1):
    
    # Get track lengths
    candidate_lengths = reconstruction_df.track_id.value_counts(sort=False)\
        .reset_index().rename(
            columns={"index":"track_id", "track_id": "n_reco_hits"})

    # Get true track lengths
    particle_lengths = reconstruction_df.drop_duplicates(subset=['hit_id']).particle_id.value_counts(sort=False)\
        .reset_index().rename(
            columns={"index":"particle_id", "particle_id": "n_true_hits"})

    spacepoint_matching = reconstruction_df.groupby(['track_id', 'particle_id']).size()\
        .reset_index().rename(columns={0:"n_shared"})

    spacepoint_matching = spacepoint_matching.merge(candidate_lengths, on=['track_id'], how='left')
    spacepoint_matching = spacepoint_matching.merge(particle_lengths, on=['particle_id'], how='left')
    # spacepoint_matching = spacepoint_matching.merge(particles_df, on=['particle_id'], how='left')

    # Filter out tracks with too few shared spacepoints
    spacepoint_matching["is_matchable"] = spacepoint_matching.n_reco_hits >= min_track_length
    spacepoint_matching["is_reconstructable"] = spacepoint_matching.n_true_hits >= min_particle_length

    return spacepoint_matching

def calculate_matching_fraction(spacepoint_matching_df):
    spacepoint_matching_df = spacepoint_matching_df.assign(
        purity_reco=np.true_divide(spacepoint_matching_df.n_shared, spacepoint_matching_df.n_reco_hits))
    spacepoint_matching_df = spacepoint_matching_df.assign(
        eff_true = np.true_divide(spacepoint_matching_df.n_shared, spacepoint_matching_df.n_true_hits))

    return spacepoint_matching_df

def evaluate_labelled_graph(graph, matching_fraction=0.5, matching_style="ATLAS", min_track_length=1, min_particle_length=1):

    if matching_fraction < 0.5:
        raise ValueError("Matching fraction must be >= 0.5")

    if matching_fraction == 0.5:
        # Add a tiny bit of noise to the matching fraction to avoid double-matched tracks
        matching_fraction += 0.00001

    # Load the labelled graphs as reconstructed dataframes
    reconstruction_df = load_reconstruction_df(graph)
    particles_df = load_particles_df(graph)

    # Get matching dataframe
    matching_df = get_matching_df(reconstruction_df, particles_df, min_track_length=min_track_length, min_particle_length=min_particle_length) 
    matching_df["event_id"] = int(graph.event_id)

    # calculate matching fraction
    matching_df = calculate_matching_fraction(matching_df)

    # Run matching depending on the matching style
    if matching_style == "ATLAS":
        matching_df["is_matched"] = matching_df["is_reconstructed"] = matching_df.purity_reco >= matching_fraction
    elif matching_style == "one_way":
        matching_df["is_matched"] = matching_df.purity_reco >= matching_fraction
        matching_df["is_reconstructed"] = matching_df.eff_true >= matching_fraction
    elif matching_style == "two_way":
        matching_df["is_matched"] = matching_df["is_reconstructed"] = (matching_df.purity_reco >= matching_fraction) & (matching_df.eff_true >= matching_fraction)

    return matching_df

In [21]:
def evaluate_labelled_graphs(graphset, config):
    all_y_truth, all_pt  = [], []

    evaluated_events = [
        utils.evaluate_labelled_graph(
            event,
            matching_fraction=config["matching_fraction"],
            matching_style=config["matching_style"],
            min_track_length=config["min_track_length"],
            min_particle_length=config["min_particle_length"],
        )
        for event in tqdm(graphset)
    ]
    evaluated_events = pd.concat(evaluated_events)

    particles = evaluated_events[evaluated_events["is_reconstructable"]]
    reconstructed_particles = particles[particles["is_reconstructed"] & particles["is_matchable"]]
    tracks = evaluated_events[evaluated_events["is_matchable"]]
    matched_tracks = tracks[tracks["is_matched"]]

    n_particles = len(particles.drop_duplicates(subset=['event_id', 'particle_id']))
    n_reconstructed_particles = len(reconstructed_particles.drop_duplicates(subset=['event_id', 'particle_id']))

    n_tracks = len(tracks.drop_duplicates(subset=['event_id', 'track_id']))
    n_matched_tracks = len(matched_tracks.drop_duplicates(subset=['event_id', 'track_id']))

    n_dup_reconstructed_particles = len(reconstructed_particles) - n_reconstructed_particles

    logging.info(f"Number of reconstructed particles: {n_reconstructed_particles}")
    logging.info(f"Number of particles: {n_particles}")
    logging.info(f"Number of matched tracks: {n_matched_tracks}")
    logging.info(f"Number of tracks: {n_tracks}")
    logging.info(f"Number of duplicate reconstructed particles: {n_dup_reconstructed_particles}")   

    # Plot the results across pT and eta
    eff = n_reconstructed_particles / n_particles
    fake_rate = 1 - (n_matched_tracks / n_tracks)
    dup_rate = n_dup_reconstructed_particles / n_reconstructed_particles

    logging.info(f"Efficiency: {eff:.3f}")
    logging.info(f"Fake rate: {fake_rate:.3f}")
    logging.info(f"Duplication rate: {dup_rate:.3f}")

In [22]:
evaluate_labelled_graphs([sample], config)

100%|██████████| 1/1 [00:00<00:00, 20.92it/s]
INFO:root:Number of reconstructed particles: 1308
INFO:root:Number of particles: 1425
INFO:root:Number of matched tracks: 1384
INFO:root:Number of tracks: 1390
INFO:root:Number of duplicate reconstructed particles: 75
INFO:root:Efficiency: 0.918
INFO:root:Fake rate: 0.004
INFO:root:Duplication rate: 0.057


### Alter to Handle Multiple Labels per Node

In [73]:
def load_reconstruction_df_new(graph, track_df):
    """Load the reconstructed tracks from a file."""
    particle_df = pd.DataFrame(
        {
            "hit_id": torch.flatten(graph.track_edges),
            "particle_id": graph.particle_id.repeat(2),
            "pt": graph.pt.repeat(2),
            "primary": graph.primary.repeat(2),
        }
    )
    particle_df = particle_df.drop_duplicates(subset=["hit_id", "particle_id"])

    return pd.merge(
        track_df,
        particle_df,
        on="hit_id",
        how="outer",
    )
        

In [86]:
def get_matching_df_new(reconstruction_df, min_track_length=1, min_particle_length=1):
    
    # Get track lengths
    candidate_lengths = reconstruction_df.drop_duplicates(subset=['hit_id', 'particle_id']).track_id.value_counts(sort=False)\
        .reset_index().rename(
            columns={"index":"track_id", "track_id": "n_reco_hits"})

    # Get true track lengths
    particle_lengths = reconstruction_df.drop_duplicates(subset=['hit_id', 'track_id']).particle_id.value_counts(sort=False)\
        .reset_index().rename(
            columns={"index":"particle_id", "particle_id": "n_true_hits"})

    spacepoint_matching = reconstruction_df.groupby(['track_id', 'particle_id']).size()\
        .reset_index().rename(columns={0:"n_shared"})

    spacepoint_matching = spacepoint_matching.merge(candidate_lengths, on=['track_id'], how='left')
    spacepoint_matching = spacepoint_matching.merge(particle_lengths, on=['particle_id'], how='left')
    # spacepoint_matching = spacepoint_matching.merge(particles_df, on=['particle_id'], how='left')

    # Filter out tracks with too few shared spacepoints
    spacepoint_matching["is_matchable"] = spacepoint_matching.n_reco_hits >= min_track_length
    spacepoint_matching["is_reconstructable"] = spacepoint_matching.n_true_hits >= min_particle_length

    return spacepoint_matching

In [88]:
graph = sample
min_track_length, min_particle_length = 3, 3
matching_fraction = 0.5
if matching_fraction == 0.5:
    # Add a tiny bit of noise to the matching fraction to avoid double-matched tracks
    matching_fraction += 0.00001

reconstruction_df = load_reconstruction_df_new(graph, track_df)
# particles_df = load_particles_df(graph)

# Get matching dataframe
matching_df = get_matching_df_new(reconstruction_df, min_track_length=min_track_length, min_particle_length=min_particle_length) 
matching_df["event_id"] = int(graph.event_id)

# calculate matching fraction
matching_df = calculate_matching_fraction(matching_df)

matching_df["is_matched"] = matching_df["is_reconstructed"] = matching_df.purity_reco >= matching_fraction

particles = matching_df[matching_df["is_reconstructable"]]
reconstructed_particles = particles[particles["is_reconstructed"] & particles["is_matchable"]]
tracks = matching_df[matching_df["is_matchable"]]
matched_tracks = tracks[tracks["is_matched"]]

n_particles = len(particles.drop_duplicates(subset=['event_id', 'particle_id']))
n_reconstructed_particles = len(reconstructed_particles.drop_duplicates(subset=['event_id', 'particle_id']))

n_tracks = len(tracks.drop_duplicates(subset=['event_id', 'track_id']))
n_matched_tracks = len(matched_tracks.drop_duplicates(subset=['event_id', 'track_id']))

n_dup_reconstructed_particles = len(reconstructed_particles) - n_reconstructed_particles

logging.info(f"Number of reconstructed particles: {n_reconstructed_particles}")
logging.info(f"Number of particles: {n_particles}")
logging.info(f"Number of matched tracks: {n_matched_tracks}")
logging.info(f"Number of tracks: {n_tracks}")
logging.info(f"Number of duplicate reconstructed particles: {n_dup_reconstructed_particles}")   

# Plot the results across pT and eta
eff = n_reconstructed_particles / n_particles
fake_rate = 1 - (n_matched_tracks / n_tracks)
dup_rate = n_dup_reconstructed_particles / n_reconstructed_particles

logging.info(f"Efficiency: {eff:.3f}")
logging.info(f"Fake rate: {fake_rate:.3f}")
logging.info(f"Duplication rate: {dup_rate:.3f}")

INFO:root:Number of reconstructed particles: 1307
INFO:root:Number of particles: 1360
INFO:root:Number of matched tracks: 1383
INFO:root:Number of tracks: 1389
INFO:root:Number of duplicate reconstructed particles: 82
INFO:root:Efficiency: 0.961
INFO:root:Fake rate: 0.004
INFO:root:Duplication rate: 0.063


In [83]:
graph = sample
min_track_length, min_particle_length = 3, 3
matching_fraction = 0.5
if matching_fraction == 0.5:
    # Add a tiny bit of noise to the matching fraction to avoid double-matched tracks
    matching_fraction += 0.00001

reconstruction_df = load_reconstruction_df_new(graph, track_df)
# particles_df = load_particles_df(graph)

# Get matching dataframe
matching_df = get_matching_df_new(reconstruction_df, min_track_length=min_track_length, min_particle_length=min_particle_length) 
matching_df["event_id"] = int(graph.event_id)

# calculate matching fraction
matching_df = calculate_matching_fraction(matching_df)

matching_df["is_matched"] = matching_df["is_reconstructed"] = matching_df.purity_reco >= matching_fraction

particles = matching_df[matching_df["is_reconstructable"]]
reconstructed_particles = particles[particles["is_reconstructed"] & particles["is_matchable"]]
tracks = matching_df[matching_df["is_matchable"]]
matched_tracks = tracks[tracks["is_matched"]]

n_particles = len(particles.drop_duplicates(subset=['event_id', 'particle_id']))
n_reconstructed_particles = len(reconstructed_particles.drop_duplicates(subset=['event_id', 'particle_id']))

n_tracks = len(tracks.drop_duplicates(subset=['event_id', 'track_id']))
n_matched_tracks = len(matched_tracks.drop_duplicates(subset=['event_id', 'track_id']))

n_dup_reconstructed_particles = len(reconstructed_particles) - n_reconstructed_particles

logging.info(f"Number of reconstructed particles: {n_reconstructed_particles}")
logging.info(f"Number of particles: {n_particles}")
logging.info(f"Number of matched tracks: {n_matched_tracks}")
logging.info(f"Number of tracks: {n_tracks}")
logging.info(f"Number of duplicate reconstructed particles: {n_dup_reconstructed_particles}")   

# Plot the results across pT and eta
eff = n_reconstructed_particles / n_particles
fake_rate = 1 - (n_matched_tracks / n_tracks)
dup_rate = n_dup_reconstructed_particles / n_reconstructed_particles

logging.info(f"Efficiency: {eff:.3f}")
logging.info(f"Fake rate: {fake_rate:.3f}")
logging.info(f"Duplication rate: {dup_rate:.3f}")

INFO:root:Number of reconstructed particles: 1307
INFO:root:Number of particles: 1360
INFO:root:Number of matched tracks: 1383
INFO:root:Number of tracks: 1389
INFO:root:Number of duplicate reconstructed particles: 82
INFO:root:Efficiency: 0.961
INFO:root:Fake rate: 0.004
INFO:root:Duplication rate: 0.063
